1001 Tracklists Scraper
=======================
A set of functions to scrape music tracklists from [1001 Tracklists](https://www.1001tracklists.com)

Import a bunch of stuff

In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import urllib3
import os
import spotipy
import spotipy.util as util
import traceback
from pprint import pprint
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
root_url = 'https://www.livetracklist.com/'

Get track data from spotify and return it

In [ ]:
client_id='6389b29d73fc4806ba5e812e678854c1'
client_secret='0b4bcd832e694aedad408b5b4a93dd5c'
ccm=util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp =spotipy.Spotify(client_credentials_manager=ccm)

def get_attrs(artist, track):
    try:
        res = sp.search(q='artist:'+artist+' track:'+track, type="track")
        track_res = res['tracks']['items'][0]
        track_id = track_res['uri']
        deets = sp.audio_features(track_id)
        camelot = get_camelot(deets[0]['key'], deets[0]['mode'])
        deets[0]['camelot'] = camelot
        return pd.Series(deets[0])
    except Exception as e:
        #print(e)
        return False
    
#get_attrs(artist='Armin Van Buuren', track="shivers")

In [ ]:
def get_camelot(key, mode):
    # index of letter is spotify pitch class - e.g. 0->c, 1->c# etc.
    tones = [ 'c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b' ]
    keys = {
        'a0': '8A',
        'a1': '11B',
        'a#0': '3A',
        'a#1': '6B',
        'b0': '10A',
        'b1': '1B',
        'c0': '5A',
        'c1': '8B',
        'c#0': '12A',
        'c#1': '3B',
        'd0': '7A',
        'd1': '10B',
        'd#0': '2A',
        'd#1': '5B',
        'e0': '9A',
        'e1': '12B',
        'f0': '4A',
        'f1': '7B',
        'f#0': '11A',
        'f#1': '2B',
        'g0': '6A',
        'g1': '9B',
        'g#0': '1A',
        'g#1': '4B',
    }
    key_letter = tones[key]
    return keys[key_letter+str(mode)]

Get a tracklist from 1001 Tracklists and write it to a CSV with Spotify track info for all songs it can find

In [ ]:
def get_tracklist(url, folder='.'):
    try:
        os.makedirs(folder, exist_ok=True)
        fname = url.split('/')[-2]

        !wget {url} -q -O {fname}
        #print("Fname: "+fname)
        soup = bs(open(fname), "lxml")
        !rm {fname}

        tracklist = pd.DataFrame(columns=['Artist(s)', 'Title', 'Release'])
        set_name = soup.find(class_="entry-title").get_text().strip()
        print(set_name)
        if os.path.exists(f'{folder}/{set_name}.csv'):
            print(f'Skipping: {set_name}, already exists...')
            return    
        for li in soup.select('#TheList')[0].find_all('li'):
            try:
                link = li.find('span', class_='listItem')
                if link is None:
                    continue
                text = link.get_text()
                artists_raw = text.split('-')[0].split(' & ')
                artists = " ".join(artists_raw) 
                artists = artists.replace('vs.', ' ').strip()
                artists = artists.replace('ft.', ' ')
                #print("Artists: "+artists)

                # title after '-' but before both label ([) and release (() and if a mashup, remove all but first song to make it easier to look up
                title = text.split('-')[1].split('[')[0].split('(')[0].split('vs.')[0]
                #print("title: "+title)

                # Releases in braces
                try:
                    release = text.split('(')[1].split(')')[0]
                except:
                    release = "unknown"
                #print("release: "+release)

                # Label in []
                try:
                    label = text.split('[')[1].split(']')[0]
                except:
                    label = "uknown"

                basic_details = pd.Series([artists, title, release], index=['Artist(s)', 'Title', 'Release'])

                #don't search spotify for unknown version, just search base name
                if release == 'unknown':
                    release = ''

                spotify_details = get_attrs(artist=artists+' '+release, track=title)
                # Try removing version if not found
                if spotify_details is False:
                    for a in artists.split(' '):
                        spotify_details = get_attrs(artist=a, track=title)
                        if spotify_details is not False:
                            break
                # Set details to none instead of false if not found, so track won't get excluded
                if spotify_details is False:
                    spotify_details = None
                    print(f'Not Found: {title} by {artists}')
                row = pd.concat([basic_details, spotify_details])
                tracklist = tracklist.append(row, ignore_index=True)
            except Exception as e:
                traceback.print_exc()
                pass
        tracklist.to_csv(f'{folder}/{set_name}.csv')
    except Exception as e:
        print(e)
        pass
#get_tracklist(url='https://www.livetracklist.com/lost-frequencies-slam-mixmarathon-xxl-ade-2018/', folder='test')

Get a whole series of tracklists, and put them in a folder

In [ ]:
def get_series_tracklists(series_url):
    for page in range(310):
        fname = str(page)
        doesnt_exist = !ls | grep fname
        if doesnt_exist != 0:
            !wget {series_url+'/page/'+str(page)} 
            soup = bs(open(fname), "lxml")
            !rm {fname}
        else:
            print(fname+" already exists, skipping...")
        main = soup.find(class_='cb-main')
        title = "LiveTracklists"
        for article in main.find_all('article'):
            link = article.find('a', href=True)
            href = link['href']
            print("href: "+href)
            get_tracklist(url=href, folder=title)

                
get_series_tracklists(series_url=root_url)

Not Found:  Fade Into Zenith  by Dannic   Avicii
href: https://www.livetracklist.com/siege-bruno-martini-tomorrowland-belgium-2018-sexy-by-nature-weekend-2/
Siege & Bruno Martini @ Tomorrowland Belgium 2018 (Sexy By Nature) (Weekend 2)
Not Found:  Darth J by Mennie Julien Sandre
href: https://www.livetracklist.com/cristoph-tomorrowland-belgium-2018/
Cristoph @ Tomorrowland Belgium 2018 (Pryda) (Weekend 2)
Not Found:  Vanquish by Cristoph
Not Found:  Front & Back by Dave Angel
Not Found:  Let Me Feel by Jeremy Olander
Not Found:  The Great Crescendo by Pig&Dan Mark Reeve
href: https://www.livetracklist.com/vini-vici-tomorrowland-belgium-2018-a-state-of-trance/
Vini Vici @ Tomorrowland Belgium 2018 (A State of Trance)
Not Found:  ID by Blastoyz
Not Found:  It's My Life  by Bon Jovi
href: https://www.livetracklist.com/klingande-tomorrowland-belgium-2018/
Klingande @ Tomorrowland Belgium 2018 (Organ of Harmony) (Weekend 2)
Not Found:  Praise You by Gamper Dadoni
href: https://www.livetrack

Traceback (most recent call last):
  File "<ipython-input-5-ec4c890a13fa>", line 30, in get_tracklist
    title = text.split('-')[1].split('[')[0].split('(')[0].split('vs.')[0]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-5-ec4c890a13fa>", line 30, in get_tracklist
    title = text.split('-')[1].split('[')[0].split('(')[0].split('vs.')[0]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-5-ec4c890a13fa>", line 30, in get_tracklist
    title = text.split('-')[1].split('[')[0].split('(')[0].split('vs.')[0]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-5-ec4c890a13fa>", line 30, in get_tracklist
    title = text.split('-')[1].split('[')[0].split('(')[0].split('vs.')[0]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-5-ec4c890a13fa>", line 30, in get_tracklist
    title = text.split('-')[1].split('[')[0].s

Adriatique @ Tomorrowland Belgium 2018 (Diynamic)
Not Found:  Fade Into You by Mathame
Not Found:  Pop It Bad by Huggatron
Not Found:  ID by Adriatique
href: https://www.livetracklist.com/jonas-blue-tomorrowland-belgium-2018-stmpd-rcrds/
Jonas Blue @ Tomorrowland Belgium 2018 (STMPD RCRDS)
href: https://www.livetracklist.com/yellow-claw-tomorrowland-belgium-2018/
Yellow Claw @ Tomorrowland Belgium 2018 (Mainstage)
Not Found:  Linza by Crisis Era   Lil Debbie
Not Found:  Kaolo Part 4 by Yellow Claw
Not Found:  Chicken by Stoltenhoff GTA
Not Found:  Let's Get Ill by DJ Snake MERCER   Jermaine Dupri
Not Found:  Don't Stop by Yellow Claw Valentino Khan
href: https://www.livetracklist.com/yellow-claw-b2b-moksi-b2b-mike-cervello-parookaville-2018/
Yellow Claw b2b Moksi b2b Mike Cervello @ ParookaVille 2018
Not Found:  Let's Get Ill by DJ Snake MERCER   Jermaine Dupri
href: https://www.livetracklist.com/tchami-x-malaa-parookaville-2018/
Tchami x Malaa @ ParookaVille 2018
href: https://www.liv